In [50]:
import pandas as pd

df = pd.read_csv("tmp/argenprop_raw_extract_2024-11-26.csv")

df.head(2)

,updated_at,location,created_at,expenses,property_url,argenprop_code,address,price,description,features
0,2024-11-26,"Departamento en Alquiler en Lomas de Nuñez, Nuñez",2024-11-26,$75.000\nexpensas,https://www.argenprop.com/departamento-en-alqu...,16519581,Deheza y Arias,USD 1.300,Departamento en alquiler - 1 Dormitorio 1 Baño...,"['60 m² cubie.', '2 dorm.', '50 años']"
1,2024-11-26,"Departamento en Alquiler en Barracas, Capital ...",2024-11-26,$30.000\nexpensas,https://www.argenprop.com/departamento-en-alqu...,8328978,"Av. Martín García 400, Piso 3",$ 320.000,ALQUILER 2 AMBIENTES AL FRENTE - BARRACAS,"['360 m² cubie.', '60 años', '3 baños']"


In [51]:
# En Location tenemos la ubicación y el barrio, vamos a separarlos
df[['neighborhood', 'location']] = df['location'].str.split(",", expand=True)

In [52]:
# Nos queda en la columna neighborhood el barrio con algo de texto extra. Removemos ese texto
df['neighborhood'] = df['neighborhood'].str.replace("Departamento en Alquiler en", "").str.strip()
df['location'] = df['location'].str.strip()

# En ocasiones vemos que la columna location tiene en realidad el barrio. Vamos a corregir esto
# para ser consistentes
df.loc[df['location'] != "Capital Federal", 'neighborhood'] = df['location']

# Luego de esto, reemplazamos los valores por capital federal. Tener en cuenta que el filtro ya viene
# de la parte de scrapping
df['location'] = "Capital Federal"
df.head(2)

,updated_at,location,created_at,expenses,property_url,argenprop_code,address,price,description,features,neighborhood
0,2024-11-26,Capital Federal,2024-11-26,$75.000\nexpensas,https://www.argenprop.com/departamento-en-alqu...,16519581,Deheza y Arias,USD 1.300,Departamento en alquiler - 1 Dormitorio 1 Baño...,"['60 m² cubie.', '2 dorm.', '50 años']",Nuñez
1,2024-11-26,Capital Federal,2024-11-26,$30.000\nexpensas,https://www.argenprop.com/departamento-en-alqu...,8328978,"Av. Martín García 400, Piso 3",$ 320.000,ALQUILER 2 AMBIENTES AL FRENTE - BARRACAS,"['360 m² cubie.', '60 años', '3 baños']",Barracas


In [53]:
# Vamos a limpiar el valor de las expensas para que nos quede un número
df['expenses'] = df['expenses'].str.replace("$", "").str.replace(".", "").str.replace("expensas", "").str.strip()

In [54]:
# Vamos a normalizar el precio a una única moneda (pesos argentinos)
# Primero separaremos el valor en dos columnas, una para la moneda y otra para el valor
df[['price_currency', 'price']] = df['price'].str.split(" ", expand=True)

In [55]:
# Normalizamos a pesos
# Tenemos algunas filas con valor None, las limpiamos
df = df.loc[df['price'].notna()]
df = df.loc[df['price'] != "None"]

# Oficial Venta
VALOR_DOLAR = 1027.5

df.loc[df['price_currency'] == 'USD', 'price'] = df['price'].str.replace(".", "", regex=False).astype(float) * VALOR_DOLAR

# Drop de price_currency
df.drop(columns=['price_currency'], inplace=True)

In [56]:
import numpy as np
# Vamos a ahora extraer las features de la columna "features"

def extract_total_area(values):
    tmp_values = eval(values)
    for value in tmp_values:
        if "m²" in value:
            return value.replace("m² cubie.", "").strip() 
    # Si no encontramos el valor. Devolvemos NaN
    return np.nan

def extract_rooms(values):
    tmp_values = eval(values)
    if "Monoam." in tmp_values:
        return str(1)
    for value in tmp_values:
        if "ambientes" in value:
            return value.replace("ambientes", "").strip()   
    # Si no encontramos el valor. Devolvemos NaN
    return np.nan

def extract_bedrooms(values):
    tmp_values = eval(values)
    for value in tmp_values:
        if "dorm" in value:
            return value.replace("dorm.", "").strip()   
    # Si no encontramos el valor. Devolvemos NaN
    return np.nan

def extract_bathrooms(values):
    tmp_values = eval(values)
    for value in tmp_values:
        if "baño" in value:
            return value.replace("baños", "").replace("baño", "").strip()   
    # Si no encontramos el valor. Devolvemos NaN
    return np.nan

def extract_antiquity(values):
    tmp_values = eval(values)
    if "A Estrenar" in tmp_values:
        return str(0)
    for value in tmp_values:
        if "años" in value:
            return value.replace("años", "").strip()   
    # Si no encontramos el valor. Devolvemos NaN
    return np.nan

df["total_area"] = df["features"].apply(extract_total_area)
df["rooms"] = df["features"].apply(extract_rooms)
df["bedrooms"] = df["features"].apply(extract_bedrooms)
df["bathrooms"] = df["features"].apply(extract_bathrooms)
df["antiquity"] = df["features"].apply(extract_antiquity)
df["garages"] = np.nan

df.drop(columns=['features'], inplace=True)
df.head(20)


,updated_at,location,created_at,expenses,property_url,argenprop_code,address,price,description,neighborhood,total_area,rooms,bedrooms,bathrooms,antiquity,garages
0,2024-11-26,Capital Federal,2024-11-26,75000,https://www.argenprop.com/departamento-en-alqu...,16519581,Deheza y Arias,1335750.0,Departamento en alquiler - 1 Dormitorio 1 Baño...,Nuñez,60,NaN,2,NaN,50,NaN
1,2024-11-26,Capital Federal,2024-11-26,30000,https://www.argenprop.com/departamento-en-alqu...,8328978,"Av. Martín García 400, Piso 3",320.000,ALQUILER 2 AMBIENTES AL FRENTE - BARRACAS,Barracas,360,NaN,NaN,3,60,NaN
2,2024-11-26,Capital Federal,2024-11-26,52372,https://www.argenprop.com/departamento-en-alqu...,16408558,Doctor Rómulo Naón 3800,350.000,Alquiler Monoambiente con balcon Saavedra CABA.,Saavedra,58,NaN,2,NaN,60,NaN
3,2024-11-26,Capital Federal,2024-11-26,75000,https://www.argenprop.com/departamento-en-alqu...,16328770,"Chile 600, Piso 9",390.000,Alquiler Amplio 2 Ambientes en San Telmo,San Telmo,40,NaN,1,NaN,40,NaN
4,2024-11-26,Capital Federal,2024-11-26,140000,https://www.argenprop.com/departamento-en-alqu...,16506753,Julian Alvarez 1100,565125.0,"Departamento en Alquiler en Palermo, Capital ...",Palermo,37,NaN,NaN,1,6,NaN
5,2024-11-26,Capital Federal,2024-11-26,NaN,https://www.argenprop.com/departamento-en-alqu...,16157002,Nicaragua 5600,873375.0,Excelente 2 amb totalmente equipado en Palermo...,Palermo,37,NaN,NaN,1,15,NaN
6,2024-11-26,Capital Federal,2024-11-26,NaN,https://www.argenprop.com/departamento-en-alqu...,16364867,Julián Álvarez 2100,822000.0,AMPLIO 2 AMBIENTES A ESTRENAR,Palermo,NaN,1,NaN,NaN,NaN,NaN
7,2024-11-26,Capital Federal,2024-11-26,120000,https://www.argenprop.com/departamento-en-alqu...,16557470,Cerrito 0,700.000,ALQUILER 3 AMBIENTES Y COCH EN SAN NICOLAS CABA,Centro,43,NaN,1,NaN,40,NaN
8,2024-11-26,Capital Federal,2024-11-26,150000,https://www.argenprop.com/departamento-en-alqu...,16424418,Avenida Santa Fe y Carranza,600.000,Departamento en alquiler - 1 Dormitorio 1 Baño...,Palermo,38,NaN,1,NaN,12,NaN
9,2024-11-26,Capital Federal,2024-11-26,NaN,https://www.argenprop.com/departamento-en-alqu...,6615061,ARTIGAS 1000,550.000,3 AMB AL FRENTE CON BALCON GAONA Y ARTIGAS BUE...,Flores,65,NaN,1,2,2 b,NaN


In [57]:
df.columns

Index(['updated_at', 'location', 'created_at', 'expenses', 'property_url',
       'argenprop_code', 'address', 'price', 'description', 'neighborhood',
       'total_area', 'rooms', 'bedrooms', 'bathrooms', 'antiquity', 'garages'],
      dtype='object')